In [85]:
import pandas as pd
import numpy as np
data = pd.read_csv('data/data.csv')
#data = pd.DataFrame.from_csv('data/data.csv')

## Remove outlier ages

In [98]:
for i in range (data['Age'].size):
    data.iloc[i]['Age'] = data.iloc[i]['Age'] - data.iloc[i]['Age']%5
    
#data['Age'] = 
#[ data[i]['Age'] - data[i]['Age']%5 for i in range(1,38805) ]
#data['Age']#.apply(np.floor(5))
#%loadpy ./scripts/filename.py

In [97]:
data.iloc[0]

,Id,Age,Rank,Year,Time,Pace,Sex
0,1,30,1263,2016,15300,584,1
1,2,60,2289,2016,18342,701,1
2,3,30,2276,2016,18274,698,0
3,4,35,670,2015,13977,533,1
4,4,35,2390,2016,18810,718,1
5,5,40,2608,2016,19714,753,0
6,5,40,3086,2014,20594,786,0
7,5,35,2455,2010,19744,753,0
8,5,35,2343,2008,19729,753,0
9,5,35,2415,2009,19647,749,0


In [100]:
data['Age'].size

38804